In [1]:
pip install imageio

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import numpy as np 
import pandas as pd 
import requests 
import json
from bs4 import BeautifulSoup as BS
import imageio
from selenium import webdriver
sys.path.append('/Users/jacoblourie')
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
import time 

In [24]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options = chrome_options)

In [5]:
#this link gets a dictionary
def get_search_data(SearchTerm,offset):
    link = f'https://api.giphy.com/v1/gifs/search?api_key=pNO36s6y7DvqG22MhOS3EdvyyM9JavcJ&q={SearchTerm}&offset=offset'
    resp = requests.get(link)
    SearchData = resp.json()
    return SearchData

In [6]:
#test run
SearchTerm = 'Falling'
SearchData = get_search_data(SearchTerm,25)

In [7]:
#now we get the mp4 link and the title
##
key = 4

#get title and mp4 links
def get_mp4link_and_title(GotRequest, key):
    orig_mp4link = str(GotRequest['data'][key]['images']['original_mp4']['mp4'])
    title = GotRequest['data'][key]['title']
    return orig_mp4link, title


In [8]:
#test run
orig_mp4, title = get_mp4link_and_title(SearchData,49)
print(f'Link: {orig_mp4}\ntitle: {title}')

Link: https://media2.giphy.com/media/l41m43qH0dTpr0kww/giphy.mp4?cid=cb54c965y885jnehs7ccl4gx7zo6k7w11rom4l7dgdsl6v9x&rid=giphy.mp4&ct=g
title: Tired Tv Show GIF by RuPaul's Drag Race S5


#### Where is the image?

it's under "article / div / a / / IMG"



In [9]:
#here we take the specific web page we want to search
def ExtractImageInBytes(mp4_link):
    
    #this section gets us a specific link that looks like this 
    driver.get(mp4_link)
    time.sleep(2)
    image_sect = driver.find_elements_by_tag_name('img')
    image_link = image_sect[0].get_attribute('src')     
    if '.svg' in image_link:
        image_link = image_sect[1].get_attribute('src') 
    image_byte_form = requests.get(image_link)
    return image_byte_form

'''NOTE - sometimes it's the first in the image_1 list, and sometimes it's the second. annoying'''

"NOTE - sometimes it's the first in the image_1 list, and sometimes it's the second. annoying"

In [14]:
#here we get an image
image_byte_form = ExtractImageInBytes(orig_mp4)

In [18]:
#this saves the file
'''This should really be saving the folder to google cloud, not to drive, folder is classification list, title of image is title from gif.'''
#precede every title with it's index. "143 Man falls into pond"
#frames could be "143 man falls into pond 12"

def SaveImage(folder, counter, title, image_byte_form):
    fname = f'{folder}/{counter} {title}.gif'
    with open(fname,"wb+") as g:
        g.write(image_byte_form.content)

In [21]:
#Test run
SaveImage('Falling',0,title,image_byte_form)

Data Management: 

#actually creating a loop to get lots of data.

#storage of the data - google cloud storage? probably Google Cloud because dominika has free tokens.
- initially, decided to store these as Gifs.

#get 10 minute tutorial from szymon on branching.



In [ ]:
import pathlib
def makefolder(SearchTerm):
    pathlib.Path(SearchTerm).mkdir(parents=False, exist_ok=True)

In [ ]:
# search_terms = ['Hugging','Fighting','Running','Watching tv','Swimming','Drinking','Rowing','Playing computer game','Cooking']
# for SearchTerm in search_terms:
#     makefolder(SearchTerm)
#try a search term
SearchTerm = "Falling"
#start a counter
counter = 0

for page in range(100):
    try:
        SearchData = get_search_data(SearchTerm,offset=page)
    except:
        print('Couldnt get search data for {offset}')
        print(f'Key: {key}, counter: {counter}, page: {page}')      
        continue
    #iterate over each key in the dict
    for key in range(50):
        #get file link and title
        try:
            orig_mp4_link, title = get_mp4link_and_title(SearchData,key)
        except:
            print(f'couldnt get orig_mp4 or title for {SearchData, key} (search term, key)')
            print(f'Key: {key}, counter: {counter}, page: {page}')
            counter+=1
            continue
        #download image
        try:
            image_byte_form = ExtractImageInBytes(orig_mp4_link)
        except:
            print(f'Couldnt extract byte form for {orig_mp4_link}')
            print(f'Key: {key}, counter: {counter}, page: {page}')      
            counter+=1
            continue
        #save image with title in relevant folder
        try:
            SaveImage(SearchTerm,counter,title,image_byte_form)
        except:
            print(f'Couldnt save {title}')
            print(f'Key: {key}, counter: {counter}, page: {page}')      
            counter+=1
            continue
        counter+=1

In [65]:
key

13

# 